# Exploratory Data Analysis (EDA)

## Data description :

The dataset used in this project is a corpus of scientific abstract associated with a human supervized label describing whether the abstract is relevant or not.

## Data loading

Here we load the data files (that are scattered in different directories), into one dataset for positives and negatives.

We also consider only instances that have a doi value (we reject None values)

The other we do is that we extract positives and negatives from the corpus with the help of the given positive data.

In [1]:
from create_ipbes_raw import *

pos_ds, neg_ds, corpus_ds=loading_pipeline_from_raw()
print("corpus columns :",corpus_ds.features)
print("positives columns :",pos_ds[0].features)
print("negatives columns :",neg_ds[0].features)

print("corpus size :",len(corpus_ds))
print("positives size :",len(pos_ds[0]))
print("negatives size :",len(neg_ds[0]))

pos_raw=get_ipbes_positives()



/home/leandre/Projects/BioMoQA_Playground/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


128
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES IAS_2352922
86
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES SUA_2344805
128
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES VA_2345372
{'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None)}
creating corpus dataset
dataset loaded
['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
1


Filter (num_proc=32): 100%|██████████| 4227131/4227131 [00:32<00:00, 131807.85 examples/s]


2
Finished positives and negatives creation pipeline
corpus columns : {'id': Value(dtype='string', id=None), 'display_name': Value(dtype='string', id=None), 'author': [{'au_id': Value(dtype='string', id=None), 'au_display_name': Value(dtype='string', id=None), 'au_orcid': Value(dtype='string', id=None), 'author_position': Value(dtype='string', id=None), 'is_corresponding': Value(dtype='bool', id=None), 'au_affiliation_raw': Value(dtype='string', id=None), 'institution_id': Value(dtype='string', id=None), 'institution_display_name': Value(dtype='string', id=None), 'institution_ror': Value(dtype='string', id=None), 'institution_country_code': Value(dtype='string', id=None), 'institution_type': Value(dtype='string', id=None), 'institution_lineage': Value(dtype='string', id=None)}], 'ab': Value(dtype='string', id=None), 'doi': Value(dtype='string', id=None), 'topics': [{'i': Value(dtype='int32', id=None), 'score': Value(dtype='float64', id=None), 'name': Value(dtype='string', id=None), 'id

In [2]:
print(pos_ds)
print(neg_ds)

[Dataset({
    features: ['doi', 'title', 'abstract'],
    num_rows: 5251
}), Dataset({
    features: ['doi', 'title', 'abstract'],
    num_rows: 6281
}), Dataset({
    features: ['doi', 'title', 'abstract'],
    num_rows: 3109
})]
[Dataset({
    features: ['title', 'abstract', 'doi'],
    num_rows: 4190054
}), Dataset({
    features: ['title', 'abstract', 'doi'],
    num_rows: 4190005
}), Dataset({
    features: ['title', 'abstract', 'doi'],
    num_rows: 4185398
})]


In [21]:
print(corpus_ds.column_names)
print(corpus_ds[0])

['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
{'id': 'https://openalex.org/W1979751022', 'display_name': 'Observations on an apparatus to illustrate the composition and resolution forces, and to measure the power obtained by certain parts of machinery operating on this principle', 'author': [{'au_id': 'https://openalex.org/A5055018967', 'au_display_name': 'W. R. Johnson', 'au_orcid': None, 'author_position': 'first', 'is_corresponding': True, 'au_affiliation_raw': '', 'institution_id': None, 'institution_display_name': None, 'institution_ror': None, 'institution_country_code': None, 'institution_type': None, 'institution_lineage': None}], 'ab': "Physical components of societies like infrastructures need biophysical resources for their construction, maintenance and use. These components, analyzed as societies' material stocks, predefine energy and raw materials and provide societal services, necessary for their functioning and for social welfare. The nexus betwe

In [26]:
print(len(pos_raw[0]["Abstract Note"]))
new_pos_raw=pos_raw[0].filter(lambda x: x["Abstract Note"] != None)
print(len(new_pos_raw))


5251
2657


In [30]:
print(corpus_ds)
print(pos_ds)

corpus_pos_ds=[]
for i in range(len(pos_ds)):
    print(i)
    abs_set=set(e.strip() for e in pos_ds[i]['abstract'] if e is not None)
    titles_set=set(e.strip() for e in pos_ds[i]['title'] if e is not None)
    dois_set=set(e.strip() for e in pos_ds[i]['doi'] if e is not None)
    if None in dois_set : dois_set.remove(None) 
    if None in titles_set : titles_set.remove(None) 
    if None in abs_set : abs_set.remove(None) 
    print("titles_set size :",len(titles_set))
    print("abs_set size :",len(abs_set))
    print("dois_set size :",len(dois_set))

    seen_abstracts=set()
    seen_titles=set()
    seen_dois=set()
    def find(batch):
        batch_bools=[]
        for j in range(len(batch['display_name'])):
            title=batch['display_name'][j]
            title=title.strip() if title is not None else None
            abstract=batch['ab'][j]
            abstract=abstract.strip() if abstract is not None else None
            doi=batch['doi'][j]
            doi=doi.strip() if doi is not None else None
            
            if (abstract in abs_set) and (abstract not in seen_abstracts):
                seen_abstracts.add(abstract)
                batch_bools.append(True)
            elif (title in titles_set) and (title not in seen_titles):
                seen_titles.add(title)
                batch_bools.append(True)
            elif (doi is not None) and(any(doi.endswith(p_doi) for p_doi in dois_set)) and (doi not in seen_dois):
                batch_bools.append(True)
                seen_dois.add(doi)
            else:
                batch_bools.append(False)
        return batch_bools
    temp_ds=corpus_ds.filter(find, batched=True, batch_size=1000,num_proc=32)
    corpus_pos_ds.append(temp_ds)
    print(temp_ds)



Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 4227131
})
[Dataset({
    features: ['doi', 'title', 'abstract', 'Language'],
    num_rows: 5251
}), Dataset({
    features: ['doi', 'title', 'abstract', 'Language'],
    num_rows: 6281
}), Dataset({
    features: ['doi', 'title', 'abstract', 'Language'],
    num_rows: 3109
})]
0
titles_set size : 5228
abs_set size : 2653
dois_set size : 3861


Filter (num_proc=32): 100%|██████████| 4227131/4227131 [01:02<00:00, 67720.70 examples/s]


Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 1161
})
1
titles_set size : 6169
abs_set size : 1368
dois_set size : 2919


Filter (num_proc=32): 100%|██████████| 4227131/4227131 [00:48<00:00, 86788.27 examples/s] 


Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 1191
})
2
titles_set size : 2945
abs_set size : 2069
dois_set size : 1892


Filter (num_proc=32): 100%|██████████| 4227131/4227131 [00:32<00:00, 131456.24 examples/s]


Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 723
})


In [ ]:
def is_weird(batch):
    batch_bools=[]
    for j in range(len(batch['title'])):
        title=batch['title'][j]
        abstract=batch['abstract'][j]
        doi=batch['doi'][j]
        if doi =='DOI':
            batch_bools.append(True)
        elif title == 'Title':
            batch_bools.append(True)
        elif abstract == 'Abstract Note':
            batch_bools.append(True)
        else:
            batch_bools.append(False)
    return batch_bools

weird_pos=pos_ds[0].filter(is_weird, batched=True, batch_size=1000,num_proc=32)
print(weird_pos)

Filter (num_proc=32): 100%|██████████| 5251/5251 [00:00<00:00, 19313.10 examples/s]

Dataset({
    features: ['doi', 'title', 'abstract'],
    num_rows: 0
})


In [ ]:
# Convert corpus_ds to a set for faster lookups
abstract_set = set(corpus_ds['ab'])
title_set = set(corpus_ds['display_name'])

# Check for matches based on Abstract
abstract_matches = []
for pos_dataset in pos_raw:
    matches = pos_dataset.filter(lambda x: x['Abstract Note'] in abstract_set)
    abstract_matches.append(matches)

# Check for matches based on Title
title_matches = []
for pos_dataset in pos_raw:
    matches = pos_dataset.filter(lambda x: x['Title'] in title_set)
    title_matches.append(matches)

# Print results
print("Matches based on abstract:")
for i, match in enumerate(abstract_matches):
    print(f"Dataset {i}: {len(match)} matches")

print("\nMatches based on Title:")
for i, match in enumerate(title_matches):
    print(f"Dataset {i}: {len(match)} matches")

NameError: name 'corpus_ds' is not defined

In [ ]:

print(pos_raw)
print("positives raw size :",[len(pos_raw[i]) for i in range(len(pos_raw))])
print("positives raw columns :",[pos_raw[i].features for i in range(len(pos_raw))])

Generating train split: 0 examples [00:00, ? examples/s]


Error loading files from /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES IAS_2352922: An error occurred while generating the dataset


Generating train split: 0 examples [00:00, ? examples/s]


Error loading files from /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES SUA_2344805: An error occurred while generating the dataset


Generating train split: 0 examples [00:00, ? examples/s]

Error loading files from /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES VA_2345372: An error occurred while generating the dataset
[]
positives raw size : []
positives raw columns : []


## Data pre-processing

Now that we have loaded the data we are able to clean them and pre-process them in order to give them to the model.

In particular what we want to do is :
- Reject abstracts with None values
- Manage duplicates
- Manage conflicts (cases wher you have an isntance in both positives and negatives)
- Consider only journal articles

In [2]:
print(pos_ds[0][0])

{'id': 'https://openalex.org/W2099622325', 'display_name': 'Habitat and Community Modification by An Introduced Herbivorous Snail', 'author': [{'au_id': 'https://openalex.org/A5033598790', 'au_display_name': 'Mark D. Bertness', 'au_orcid': None, 'author_position': 'first', 'is_corresponding': True, 'au_affiliation_raw': '', 'institution_id': None, 'institution_display_name': None, 'institution_ror': None, 'institution_country_code': None, 'institution_type': None, 'institution_lineage': None}], 'ab': 'Experimental removal of the introduced herbivorous snail Littorina littorea from a protected New England rocky beach resulted in rapid habitat and community changes. At normal snail densities, L. littorea grazing bulldozes sediments from hard substrate and precludes the presence of an algal canopy. Snail removal resulted in rapid sediment accumulation and the development of an algal canopy, which accelerated sedimentation and bound sediment to hard substrate. These changes led to the incr

## IAS :

In [1]:
from preprocess_ipbes import *

data_dict=data_pipeline()

print(data_dict)

/home/leandre/Projects/BioMoQA_Playground/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


128
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES IAS_2352922
86
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES SUA_2344805
128
Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/corpus/Raw/Positives/IPBES VA_2345372
{'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None)}
creating corpus dataset
dataset loaded
['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
1
2
Finished positives and negatives creation pipeline
Processing dataset for type: IAS
Positive dataset size: 5251
Negative dataset size: 4190054
pos_ds size: 5251
neg_ds size: 4190054
Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4195305
})
5251
Filtering out rows with no abstracts or DOI...
Size of the dataset

Filter (num_proc=32): 100%|██████████| 4195305/4195305 [00:04<00:00, 952645.45 examples/s] 


Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4114546
})
Processing dataset for type: SUA
Positive dataset size: 6281
Negative dataset size: 4190005
pos_ds size: 6281
neg_ds size: 4190005
Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4196286
})
6281
Filtering out rows with no abstracts or DOI...
Size of the dataset before cleaning: 4196286
Applying clean_filter...


Filter (num_proc=32): 100%|██████████| 4196286/4196286 [00:05<00:00, 780807.76 examples/s] 


Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4113318
})
Processing dataset for type: VA
Positive dataset size: 3109
Negative dataset size: 4185398
pos_ds size: 3109
neg_ds size: 4185398
Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4188507
})
3109
Filtering out rows with no abstracts or DOI...
Size of the dataset before cleaning: 4188507
Applying clean_filter...


Filter (num_proc=32): 100%|██████████| 4188507/4188507 [00:04<00:00, 864190.78 examples/s] 


Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4109595
})
Completed processing all datasets.
{'IAS': (Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 2880182
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 1049209
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 185155
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4114546
})), 'SUA': (Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 2879322
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 1048896
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 185100
}), Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 4113318
})), 'VA': (Dataset({
    features: ['doi', 'title', 'abstract', 'labels'],
    num_rows: 2876716
}), Dataset({
    features: ['doi', 'title', '